In [39]:
import pandas as pd
import json
from tqdm import tqdm

In [40]:
data = pd.read_csv("../assets/spotify_data.csv")

In [41]:
with open("../assets/music_genres_tree.json", 'r') as f:
	tree = json.load(f)

In [42]:
def get_leafs(tree):
    childs = tree.get("children")
    if childs is None:
        songs_list = []
        tree["songs"] = songs_list
        yield (tree["name"], songs_list)
        return
    for child in childs:
        yield from get_leafs(child)

In [43]:
leaves = dict([*get_leafs(tree)])

In [44]:
assert (expected_keys:=set(['acoustic', 'afrobeat', 'alt-rock', 'ambient', 'black-metal',
       'blues', 'breakbeat', 'cantopop', 'chicago-house', 'chill',
       'classical', 'club', 'comedy', 'country', 'dance', 'dancehall',
       'death-metal', 'deep-house', 'detroit-techno', 'disco',
       'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic',
       'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german',
       'gospel', 'goth', 'grindcore', 'groove', 'guitar', 'hard-rock',
       'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'house',
       'indian', 'indie-pop', 'industrial', 'jazz', 'k-pop', 'metal',
       'metalcore', 'minimal-techno', 'new-age', 'opera', 'party',
       'piano', 'pop', 'pop-film', 'power-pop', 'progressive-house',
       'psych-rock', 'punk', 'punk-rock', 'rock', 'rock-n-roll',
       'romance', 'sad', 'salsa', 'samba', 'sertanejo', 'show-tunes',
       'singer-songwriter', 'ska', 'sleep', 'songwriter', 'soul',
       'spanish', 'swedish', 'tango', 'techno', 'trance', 'trip-hop'])) == set(leaves.keys())

In [45]:
len(dict([*get_leafs(tree)])), len(expected_keys)

(82, 82)

In [46]:
data.columns

Index(['Unnamed: 0', 'artist_name', 'track_name', 'track_id', 'popularity',
       'year', 'genre', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')

In [47]:
data.head()[["track_name", "track_id"]]

,track_name,track_id
0,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6
1,93 Million Miles,1s8tP3jP4GZcyHDsjvw218
2,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F
3,Fast Car,63wsZUhUZLlh1OsyrZq7sz
4,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8


In [48]:
corrected_data = data.copy()
corrected_data['track_name'] = corrected_data['track_name'].fillna("Unknown Track")

print(f"Utilisation des données corrigées: {len(corrected_data)} lignes")

leaves = dict([*get_leafs(tree)])
for song in tqdm(corrected_data.iterrows(), total=len(corrected_data)):
    leaves[dict(song[1])["genre"]].append(song[1][["track_name", "track_id"]])

Utilisation des données corrigées: 1159764 lignes


100%|██████████| 1159764/1159764 [14:05<00:00, 1371.79it/s]


In [49]:
# Convertir les Series en dictionnaires 
leaves_serializable = {}
for genre, songs_list in tqdm(leaves.items(), desc="Converting to serializable format"):
    leaves_serializable[genre] = [song.to_dict() for song in songs_list]

Converting to serializable format: 100%|██████████| 82/82 [00:20<00:00,  4.05it/s]


In [50]:
# Maintenant créons la structure hiérarchique avec l'arbre des genres enrichi
def enrich_tree_with_songs(tree_node, songs_by_genre):
    """Enrichit récursivement l'arbre avec les chansons"""
    # Ajouter les chansons pour ce nœud
    tree_node['songs'] = songs_by_genre.get(tree_node['name'], [])
    
    # Si c'est une feuille, pas d'enfants à traiter
    if 'children' not in tree_node or not tree_node['children']:
        return tree_node
    
    # Traiter récursivement les enfants
    for child in tree_node['children']:
        enrich_tree_with_songs(child, songs_by_genre)
    
    # Agréger les chansons des enfants dans le parent
    all_child_songs = []
    for child in tree_node['children']:
        all_child_songs.extend(child.get('songs', []))
    
    # Combiner les chansons directes et des enfants (sans doublons)
    seen_ids = set()
    combined_songs = []
    
    for song in tree_node['songs'] + all_child_songs:
        if song['track_id'] not in seen_ids:
            seen_ids.add(song['track_id'])
            combined_songs.append(song)
    
    tree_node['songs'] = combined_songs
    return tree_node

# Enrichir l'arbre avec les chansons
enriched_tree = enrich_tree_with_songs(tree.copy(), leaves_serializable)

print(f"Arbre enrichi créé avec {len(enriched_tree['songs'])} chansons au niveau racine")

Arbre enrichi créé avec 1159764 chansons au niveau racine


# Calcul des métriques agrégées pour chaque nœud

Maintenant que l'arbre est enrichi avec les chansons, nous allons calculer les métriques agrégées (moyennes, médianes, min/max) pour chaque nœud afin de pouvoir les utiliser dans la visualisation interactive.

In [51]:
def calculate_metrics_for_songs(songs_list):
    """Calcule les métriques agrégées pour une liste de chansons"""
    if not songs_list:
        return {
            "count": 0,
            "avg_danceability": 0,
            "avg_energy": 0,
            "avg_popularity": 0,
            "avg_duration": 0,
            "median_year": 0,
            "year_range": [0, 0],
            "min_tempo": 0,
            "max_tempo": 0,
            "avg_valence": 0
        }
    
    # Récupérer les données complètes depuis le CSV pour chaque chanson
    song_track_ids = [song['track_id'] for song in songs_list]
    
    # Filtrer les données du CSV pour les chansons de ce genre
    genre_data = corrected_data[corrected_data['track_id'].isin(song_track_ids)]
    
    if len(genre_data) == 0:
        return {
            "count": len(songs_list),
            "avg_danceability": 0,
            "avg_energy": 0,
            "avg_popularity": 0,
            "avg_duration": 0,
            "median_year": 0,
            "year_range": [0, 0],
            "min_tempo": 0,
            "max_tempo": 0,
            "avg_valence": 0
        }
    
    # Calculer les métriques
    metrics = {
        "count": len(songs_list),
        "avg_danceability": round(genre_data['danceability'].mean(), 3),
        "avg_energy": round(genre_data['energy'].mean(), 3),
        "avg_popularity": round(genre_data['popularity'].mean(), 1),
        "avg_duration": round(genre_data['duration_ms'].mean(), 0),
        "median_year": int(genre_data['year'].median()),
        "year_range": [int(genre_data['year'].min()), int(genre_data['year'].max())],
        "min_tempo": round(genre_data['tempo'].min(), 1),
        "max_tempo": round(genre_data['tempo'].max(), 1),
        "avg_valence": round(genre_data['valence'].mean(), 3)
    }
    
    return metrics

print("✅ Fonction calculate_metrics_for_songs définie")

✅ Fonction calculate_metrics_for_songs définie


In [52]:
def enrich_tree_with_metrics(tree_node):
    """Enrichit récursivement l'arbre avec les métriques pour chaque nœud"""
    
    # Calculer les métriques pour les chansons de ce nœud
    if 'songs' in tree_node and tree_node['songs']:
        tree_node['metrics'] = calculate_metrics_for_songs(tree_node['songs'])
    else:
        tree_node['metrics'] = calculate_metrics_for_songs([])
    
    # Traiter récursivement les enfants
    if 'children' in tree_node and tree_node['children']:
        for child in tree_node['children']:
            enrich_tree_with_metrics(child)
    
    return tree_node

print("✅ Fonction enrich_tree_with_metrics définie")

✅ Fonction enrich_tree_with_metrics définie


In [53]:
# Enrichir l'arbre avec les métriques
print("🔄 Enrichissement de l'arbre avec les métriques...")

# Appliquer l'enrichissement des métriques à l'arbre complet
enriched_tree_with_metrics = enrich_tree_with_metrics(enriched_tree.copy())

print(f"✅ Arbre enrichi avec métriques créé")
print(f"📊 Métriques au niveau racine:")
print(f"   - Nombre total de chansons: {enriched_tree_with_metrics['metrics']['count']}")
print(f"   - Danceability moyenne: {enriched_tree_with_metrics['metrics']['avg_danceability']}")
print(f"   - Energy moyenne: {enriched_tree_with_metrics['metrics']['avg_energy']}")
print(f"   - Popularité moyenne: {enriched_tree_with_metrics['metrics']['avg_popularity']}")
print(f"   - Plage d'années: {enriched_tree_with_metrics['metrics']['year_range']}")

# Afficher quelques métriques des enfants du premier niveau
if 'children' in enriched_tree_with_metrics and enriched_tree_with_metrics['children']:
    print(f"\n📊 Exemples de métriques des genres principaux:")
    for i, child in enumerate(enriched_tree_with_metrics['children'][:3]):
        metrics = child['metrics']
        print(f"   {child['name']}: {metrics['count']} chansons, danceability={metrics['avg_danceability']}, energy={metrics['avg_energy']}")
    if len(enriched_tree_with_metrics['children']) > 3:
        print(f"   ... et {len(enriched_tree_with_metrics['children']) - 3} autres genres")

🔄 Enrichissement de l'arbre avec les métriques...
✅ Arbre enrichi avec métriques créé
📊 Métriques au niveau racine:
   - Nombre total de chansons: 1159764
   - Danceability moyenne: 0.537
   - Energy moyenne: 0.64
   - Popularité moyenne: 18.4
   - Plage d'années: [2000, 2023]

📊 Exemples de métriques des genres principaux:
   Traditions & Folk: 106719 chansons, danceability=0.575, energy=0.579
   Classical & Soundtrack: 75130 chansons, danceability=0.434, energy=0.35
   Jazz, Funk & Soul: 81356 chansons, danceability=0.568, energy=0.585
   ... et 7 autres genres
✅ Arbre enrichi avec métriques créé
📊 Métriques au niveau racine:
   - Nombre total de chansons: 1159764
   - Danceability moyenne: 0.537
   - Energy moyenne: 0.64
   - Popularité moyenne: 18.4
   - Plage d'années: [2000, 2023]

📊 Exemples de métriques des genres principaux:
   Traditions & Folk: 106719 chansons, danceability=0.575, energy=0.579
   Classical & Soundtrack: 75130 chansons, danceability=0.434, energy=0.35
   Jazz

In [54]:
output_file = "../assets/indexByGenreSongs.json"
with tqdm(total=1, desc="Saving enriched tree with metrics") as pbar:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(enriched_tree_with_metrics, f, indent=2, ensure_ascii=False)
    pbar.update(1)

print(f"Structure hiérarchique avec métriques sauvegardée dans {output_file}")
print("Le fichier contient maintenant:")
print("- Structure arborescente compatible avec TreeVizProcessor")
print("- Chansons indexées par genre")
print("- Métriques agrégées pour chaque nœud (danceability, energy, popularity, etc.)")
print("✅ Prêt pour la visualisation interactive avec filtres !")

Saving enriched tree with metrics: 100%|██████████| 1/1 [00:47<00:00, 47.64s/it]

Structure hiérarchique avec métriques sauvegardée dans ../assets/indexByGenreSongs.json
Le fichier contient maintenant:
- Structure arborescente compatible avec TreeVizProcessor
- Chansons indexées par genre
- Métriques agrégées pour chaque nœud (danceability, energy, popularity, etc.)
✅ Prêt pour la visualisation interactive avec filtres !
